# LSTM tutorial
* Write up: https://medium.com/towards-data-science/lstm-by-example-using-tensorflow-feb0c1968537
* Code: https://github.com/roatienza/Deep-Learning-Experiments/blob/master/Experiments/Tensorflow/RNN/rnn_words.py

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import pandas as pd

A Recurrent Neural Network (LSTM) implementation example using TensorFlow..
Next word prediction after n_input words learned from text file.
A story is automatically generated if the predicted word is fed back as input.
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments

![](https://cdn-images-1.medium.com/max/1600/1*XAJdt_EbedqDlrTT9eqWvQ.png)

## Get training data
<div class=warn>
**TODO:** DO WE NEED TO NORMALIZE??

Assuming that the final output goes through a softmax or tanh layer, our current input would not make any sense
<div/>

In [2]:
df = pd.read_csv('rnn_input_test.csv', engine='c')
dfin = df.loc[df['author'] == 'I_love_Tanya_Romanova']
dfin.drop(labels=['author', 'startTimeSeconds', 'userMinutes'], axis=1, inplace=True)
dfin['points']/= max(dfin['points'])
dfin['problemRating']/= 4000
dfin.head()

X = np.array(dfin)
# chop off the last point in X and assign it to Y
#Y = X[1:]
#X = X[0:-1]
#print X.shape
#print Y.shape

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

# MNIST TEMPLATE
https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py

In [3]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 1
display_step = 100

# Network Parameters
# shape of X and Y: (10108, 21)

n_input = 21 # MNIST data input (img shape: 28*28)
n_steps = 1 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 21 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
offset = 0
# keep track of last n accuracies
widx = 0
windowsize = display_step
accwindow = [0 for i in range(windowsize)]

with tf.Session() as sess:
    sess.run(init)
    step = 0
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        #batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = np.reshape(X[offset], [1, -1])
        batch_y = np.reshape(X[offset+1], [1, -1])
        offset = (offset+1)%(X.shape[0]-1)
        
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        
        accwindow[widx] = (sess.run(accuracy, feed_dict={x: batch_x, y: batch_y}))
        widx = (widx+1)%windowsize
        
        if step % display_step == 0:
            # Calculate batch accuracy
            #acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            acc = np.mean(accwindow)
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

#    # Calculate accuracy for 128 mnist test images
#    test_len = 128
#    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
#    test_label = mnist.test.labels[:test_len]
#    print("Testing Accuracy:", \
#        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
#

Iter 0, Minibatch Loss= 7.467062, Training Accuracy= 0.00000
Iter 100, Minibatch Loss= 5.178131, Training Accuracy= 0.27000
Iter 200, Minibatch Loss= 3.237457, Training Accuracy= 0.33000
Iter 300, Minibatch Loss= 9.151541, Training Accuracy= 0.85000
Iter 400, Minibatch Loss= 3.203139, Training Accuracy= 0.76000
Iter 500, Minibatch Loss= 5.851834, Training Accuracy= 0.39000
Iter 600, Minibatch Loss= 24.459435, Training Accuracy= 0.31000
Iter 700, Minibatch Loss= 2.238639, Training Accuracy= 0.63000
Iter 800, Minibatch Loss= 1.519399, Training Accuracy= 0.56000
Iter 900, Minibatch Loss= 2.013444, Training Accuracy= 0.89000
Iter 1000, Minibatch Loss= 9.683973, Training Accuracy= 0.56000
Iter 1100, Minibatch Loss= 2.648934, Training Accuracy= 0.90000
Iter 1200, Minibatch Loss= 3.135357, Training Accuracy= 0.90000
Iter 1300, Minibatch Loss= 4.568818, Training Accuracy= 0.92000
Iter 1400, Minibatch Loss= 1.985493, Training Accuracy= 0.90000
Iter 1500, Minibatch Loss= 10.942942, Training Accu

Iter 12700, Minibatch Loss= 373.386017, Training Accuracy= 0.74000
Iter 12800, Minibatch Loss= 7.558991, Training Accuracy= 0.58000
Iter 12900, Minibatch Loss= 6.385266, Training Accuracy= 0.35000
Iter 13000, Minibatch Loss= 12.274896, Training Accuracy= 0.62000
Iter 13100, Minibatch Loss= 5.697628, Training Accuracy= 0.51000
Iter 13200, Minibatch Loss= 2.373706, Training Accuracy= 0.59000
Iter 13300, Minibatch Loss= 2.056943, Training Accuracy= 0.65000
Iter 13400, Minibatch Loss= 366.327087, Training Accuracy= 0.70000
Iter 13500, Minibatch Loss= 66.831718, Training Accuracy= 0.43000
Iter 13600, Minibatch Loss= 367.310547, Training Accuracy= 0.74000
Iter 13700, Minibatch Loss= 362.950958, Training Accuracy= 0.52000
Iter 13800, Minibatch Loss= 63.097336, Training Accuracy= 0.44000
Iter 13900, Minibatch Loss= 352.129761, Training Accuracy= 0.68000
Iter 14000, Minibatch Loss= 359.953186, Training Accuracy= 0.57000
Iter 14100, Minibatch Loss= 3.464021, Training Accuracy= 0.46000
Iter 14200

Iter 25300, Minibatch Loss= 575.297485, Training Accuracy= 0.31000
Iter 25400, Minibatch Loss= 5.239376, Training Accuracy= 0.27000
Iter 25500, Minibatch Loss= 900.296997, Training Accuracy= 0.16000
Iter 25600, Minibatch Loss= 4.255236, Training Accuracy= 0.47000
Iter 25700, Minibatch Loss= 1.709249, Training Accuracy= 0.43000
Iter 25800, Minibatch Loss= 141.318634, Training Accuracy= 0.10000
Iter 25900, Minibatch Loss= 2.453186, Training Accuracy= 0.04000
Iter 26000, Minibatch Loss= 3.290142, Training Accuracy= 0.24000
Iter 26100, Minibatch Loss= 4.966798, Training Accuracy= 0.49000
Iter 26200, Minibatch Loss= 1.806558, Training Accuracy= 0.32000
Iter 26300, Minibatch Loss= 11.932815, Training Accuracy= 0.41000
Iter 26400, Minibatch Loss= 1.771904, Training Accuracy= 0.33000
Iter 26500, Minibatch Loss= 3.297170, Training Accuracy= 0.75000
Iter 26600, Minibatch Loss= 1.974526, Training Accuracy= 0.61000
Iter 26700, Minibatch Loss= 5.761121, Training Accuracy= 0.55000
Iter 26800, Miniba

Iter 37900, Minibatch Loss= 1.815723, Training Accuracy= 0.88000
Iter 38000, Minibatch Loss= 2.033239, Training Accuracy= 0.48000
Iter 38100, Minibatch Loss= 12.132668, Training Accuracy= 0.51000
Iter 38200, Minibatch Loss= 2.350917, Training Accuracy= 0.82000
Iter 38300, Minibatch Loss= 3.524804, Training Accuracy= 0.53000
Iter 38400, Minibatch Loss= 2.776789, Training Accuracy= 0.71000
Iter 38500, Minibatch Loss= 8.233222, Training Accuracy= 0.51000
Iter 38600, Minibatch Loss= 4.752563, Training Accuracy= 0.24000
Iter 38700, Minibatch Loss= 1570.190186, Training Accuracy= 0.58000
Iter 38800, Minibatch Loss= 1.578895, Training Accuracy= 0.21000
Iter 38900, Minibatch Loss= 127.249054, Training Accuracy= 0.30000
Iter 39000, Minibatch Loss= 825.162598, Training Accuracy= 0.73000
Iter 39100, Minibatch Loss= 518.590027, Training Accuracy= 0.44000
Iter 39200, Minibatch Loss= 1.697605, Training Accuracy= 0.28000
Iter 39300, Minibatch Loss= 140.151947, Training Accuracy= 0.44000
Iter 39400, M

Iter 50400, Minibatch Loss= 5.130312, Training Accuracy= 0.69000
Iter 50500, Minibatch Loss= 1.991751, Training Accuracy= 0.69000
Iter 50600, Minibatch Loss= 6.315299, Training Accuracy= 0.03000
Iter 50700, Minibatch Loss= 3.078379, Training Accuracy= 0.27000
Iter 50800, Minibatch Loss= 2.834531, Training Accuracy= 0.63000
Iter 50900, Minibatch Loss= 146.483612, Training Accuracy= 0.94000
Iter 51000, Minibatch Loss= 37.183990, Training Accuracy= 0.56000
Iter 51100, Minibatch Loss= 663.615112, Training Accuracy= 0.00000
Iter 51200, Minibatch Loss= 136.483353, Training Accuracy= 0.00000
Iter 51300, Minibatch Loss= 2.540080, Training Accuracy= 0.43000
Iter 51400, Minibatch Loss= 3.615275, Training Accuracy= 0.90000
Iter 51500, Minibatch Loss= 14.111139, Training Accuracy= 0.75000
Iter 51600, Minibatch Loss= 2.542700, Training Accuracy= 0.65000
Iter 51700, Minibatch Loss= 2.946101, Training Accuracy= 0.93000
Iter 51800, Minibatch Loss= 1319.208252, Training Accuracy= 0.95000
Iter 51900, Mi

Iter 63000, Minibatch Loss= 4.927192, Training Accuracy= 0.33000
Iter 63100, Minibatch Loss= 204.362732, Training Accuracy= 0.73000
Iter 63200, Minibatch Loss= 785.437988, Training Accuracy= 0.54000
Iter 63300, Minibatch Loss= 3.998757, Training Accuracy= 0.67000
Iter 63400, Minibatch Loss= 778.888062, Training Accuracy= 0.15000
Iter 63500, Minibatch Loss= 1286.313843, Training Accuracy= 0.42000
Iter 63600, Minibatch Loss= 22.390844, Training Accuracy= 0.46000
Iter 63700, Minibatch Loss= 3.694134, Training Accuracy= 0.52000
Iter 63800, Minibatch Loss= 4.502112, Training Accuracy= 0.65000
Iter 63900, Minibatch Loss= 217.908081, Training Accuracy= 0.59000
Iter 64000, Minibatch Loss= 2006.485596, Training Accuracy= 0.56000
Iter 64100, Minibatch Loss= 804.594604, Training Accuracy= 0.52000
Iter 64200, Minibatch Loss= 3.926171, Training Accuracy= 0.59000
Iter 64300, Minibatch Loss= 2.381896, Training Accuracy= 0.55000
Iter 64400, Minibatch Loss= 18.169849, Training Accuracy= 0.42000
Iter 64

Iter 75400, Minibatch Loss= 937.328857, Training Accuracy= 0.88000
Iter 75500, Minibatch Loss= 2.805535, Training Accuracy= 0.60000
Iter 75600, Minibatch Loss= 1.485050, Training Accuracy= 0.51000
Iter 75700, Minibatch Loss= 251.831818, Training Accuracy= 0.22000
Iter 75800, Minibatch Loss= 3.658755, Training Accuracy= 0.16000
Iter 75900, Minibatch Loss= 931.513794, Training Accuracy= 0.11000
Iter 76000, Minibatch Loss= 2.566534, Training Accuracy= 0.14000
Iter 76100, Minibatch Loss= 1.855499, Training Accuracy= 0.22000
Iter 76200, Minibatch Loss= 1531.987549, Training Accuracy= 0.65000
Iter 76300, Minibatch Loss= 5.786345, Training Accuracy= 0.13000
Iter 76400, Minibatch Loss= 253.112854, Training Accuracy= 0.03000
Iter 76500, Minibatch Loss= 1530.530029, Training Accuracy= 0.12000
Iter 76600, Minibatch Loss= 930.382019, Training Accuracy= 0.44000
Iter 76700, Minibatch Loss= 8.466926, Training Accuracy= 0.36000
Iter 76800, Minibatch Loss= 964.323730, Training Accuracy= 0.40000
Iter 76

Iter 87800, Minibatch Loss= 3.300407, Training Accuracy= 0.54000
Iter 87900, Minibatch Loss= 1.583259, Training Accuracy= 0.24000
Iter 88000, Minibatch Loss= 297.040344, Training Accuracy= 0.29000
Iter 88100, Minibatch Loss= 1382.873535, Training Accuracy= 0.03000
Iter 88200, Minibatch Loss= 1297.154053, Training Accuracy= 0.43000
Iter 88300, Minibatch Loss= 5.566849, Training Accuracy= 0.54000
Iter 88400, Minibatch Loss= 1.800264, Training Accuracy= 0.42000
Iter 88500, Minibatch Loss= 270.534363, Training Accuracy= 0.71000
Iter 88600, Minibatch Loss= 2.216868, Training Accuracy= 0.61000
Iter 88700, Minibatch Loss= 6.988415, Training Accuracy= 0.50000
Iter 88800, Minibatch Loss= 1080.849121, Training Accuracy= 0.74000
Iter 88900, Minibatch Loss= 2.143895, Training Accuracy= 0.47000
Iter 89000, Minibatch Loss= 2.081427, Training Accuracy= 0.74000
Iter 89100, Minibatch Loss= 2733.927490, Training Accuracy= 0.18000
Iter 89200, Minibatch Loss= 16.502977, Training Accuracy= 0.48000
Iter 893

In [4]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./custom.css', "r").read()
    return HTML(styles)
css_styling()